<a href="https://colab.research.google.com/github/smitmirani10/LDATopicModelling/blob/main/Draft_project_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install torch
!pip install transformers

In [5]:
from transformers import AutoModel,AutoTokenizer
from sklearn.preprocessing import normalize
import pandas as pd
import numpy as np
import torch

In [6]:
#reading a CoNLL file into a dataframe
#DF has 4 columns, 2 from the conll file and rest from the tokens and embeddings that we will generate
#1st 2 columns are the actual word and the coreference
#3rd & 4th words are the tokens generated from that word & their corresponding embeddings generated by BERT
file1 = open('/content/temp.conll', 'r')
count = 0
sentence_list =[]
coref_list=[]
token_list=[]
embedding_list=[]
column_names = ["words", "coref","tokens_per_word","embedding_per_token"]
df = pd.DataFrame(columns = column_names)
while True:
    count += 1
    # Get next line from file
    line = file1.readline()
    #zero out the tab count before starting to read every line
    tab_count=0
    # if line is empty
    # end of file is reached
    if not line:
        break
    list_of_words = line.split()
    sentence_list.append(list_of_words[3])
    coref_list.append(list_of_words[12])
    token_list.append(None)
    embedding_list.append(None)
file1.close()
df['words'] = sentence_list
df['coref'] = coref_list
df['tokens_per_word'] = token_list
df['embedding_per_token'] = embedding_list
print(df)

             words   coref tokens_per_word embedding_per_token
0              Two      (3            None                None
1           recent       -            None                None
2           papers      3)            None                None
3          provide       -            None                None
4              new       -            None                None
5         evidence       -            None                None
6         relevant       -            None                None
7               to       -            None                None
8              the       -            None                None
9             role       -            None                None
10              of       -            None                None
11             the      (1            None                None
12          breast      (4            None                None
13          cancer       -            None                None
14  susceptibility       -            None             

In [7]:
x = df.index[df['words'].str.contains('b', case=False)]
x

Int64Index([12, 14, 16], dtype='int64')

In [8]:
#Transfer learning step
# Loading the pre-trained BERT model
###################################
# Embeddings will be derived from
# the outputs of this model
model = AutoModel.from_pretrained('microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext',
                                  output_hidden_states = True,
                                  )

# Setting up the tokenizer
###################################
# This is the same tokenizer that
# was used in the model to generate 
# embeddings to ensure consistency
tokenizer = AutoTokenizer.from_pretrained('microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext')

Downloading:   0%|          | 0.00/385 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/221k [00:00<?, ?B/s]

In [9]:
def bert_text_preparation(text, tokenizer):
    #Ref. : https://towardsdatascience.com/3-types-of-contextualized-word-embeddings-from-bert-using-transfer-learning-81fcefe3fe6d
    marked_text = "[CLS] " + text + " [SEP]"
    tokenized_text = tokenizer.tokenize(marked_text)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    segments_ids = [1]*len(indexed_tokens)
    # Convert inputs to PyTorch tensors
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])

    return tokenized_text, tokens_tensor, segments_tensors

In [10]:
def get_bert_embeddings(tokens_tensor, segments_tensors, model):
#Ref. : https://towardsdatascience.com/3-types-of-contextualized-word-embeddings-from-bert-using-transfer-learning-81fcefe3fe6d  
    with torch.no_grad():
        outputs = model(tokens_tensor, segments_tensors)
        # Removing the first hidden state
        # The first state is the input state
        hidden_states = outputs[2][1:]
    # Getting embeddings from the final BERT layer
    token_embeddings = hidden_states[-1]
    # Collapsing the tensor into 1-dimension
    token_embeddings = torch.squeeze(token_embeddings, dim=0)
    # Converting torchtensors to lists
    list_token_embeddings = [token_embed.tolist() for token_embed in token_embeddings]
    return list_token_embeddings

In [30]:
df['coref'][0]='(4'
df['coref'][2]='4)'
df

,words,coref,tokens_per_word,embedding_per_token
0,Two,(4,two,"[-0.2246626764535904, -0.26710209250450134, -0..."
1,recent,-,recent,"[-0.12273484468460083, 0.1627027690410614, -0...."
2,papers,4),papers,"[-0.1778944730758667, -0.008070876821875572, -..."
3,provide,-,provide,"[-0.035068269819021225, 0.23743653297424316, 0..."
4,new,-,new,"[-0.0618634968996048, -0.20260579884052277, 0...."
5,evidence,-,evidence,"[-0.20193272829055786, -0.029545241966843605, ..."
6,relevant,-,relevant,"[0.18042421340942383, -0.2202548384666443, -0...."
7,to,-,to,"[0.0592212937772274, -0.239136204123497, 0.023..."
8,the,-,the,"[0.10987842082977295, -0.14930360019207, 0.050..."
9,role,-,role,"[0.007047844119369984, -0.12817950546741486, 0..."


In [31]:
texts=[]
final_sentence = " ".join(df['words'].tolist())
final_sentence
texts.append(final_sentence)

In [32]:
final_sentence

'Two recent papers provide new evidence relevant to the role of the breast cancer susceptibility gene BRCA2 in DNA repair .'

In [33]:
for text in texts:
  #reading one sentence at a time and generating its tokens
  #as well as token embeddings from above methods
  tokenized_text, tokens_tensor, segments_tensors = bert_text_preparation(text, tokenizer)
  list_token_embeddings = get_bert_embeddings(tokens_tensor, segments_tensors, model)

#here we extract the tokens,except the sep and cls and punctuations
#and then we match which token is a substring of which cell
#of the word column in original DF
#when we find a match, we record the index of that item in DF
#then we update the token & embedding column with the tokens & embeddings obtained from that word
for i in tokenized_text:
  if i != "[CLS]" and i != "[SEP]" and i != ".":
    word_index = tokenized_text.index(i)
    x = df.index[df['words'].str.contains(i, case=False)]
    for j in (0,len(x)-1):
      word_embedding = list_token_embeddings[word_index]
      #print(x[j])
      df['tokens_per_word'][x[j]] = i
      df['embedding_per_token'][x[j]]=word_embedding

In [34]:
#method to read the next occurance of the coref no. for eg. 3
#if it finds that number again, it will add 
#all the words that have occured from start to end to a list
#then pass that list as a return parameter
def add_to_dict(i,number,my_coref_list,my_word_list,df):
  #print('number is',number,'location in df is',i)
  temp_coref = df['coref'][i+1]
  my_coref_list.append(df['embedding_per_token'][i+1])
  my_word_list.append(df['tokens_per_word'][i+1])
  i=i+1
  while number not in temp_coref:
    my_coref_list.append(df['embedding_per_token'][i+1])
    my_word_list.append(df['tokens_per_word'][i+1])
    temp_coref = df['coref'][i+1]
    i = i+1
  return my_coref_list,my_word_list

In [35]:
#Main purpose is to have a dictionary created for all the mentions
i=0
#coref_groups = {}
word_groups={}
mentions=[]
counter=0
#mean_vector=[]
column_names = ["index", "mention_no","norm_mean_vector","tokens"]
final_df = pd.DataFrame(columns = column_names)
for i in range(df['coref'].size):
  temp = df['coref'][i]
  mentions = temp.split('|')

  #print(mentions)
  for mention in mentions:
    #This is the case 1 where the mention
    #only has a opening and no closing and longer span
    if '(' in mention and ')' not in mention:
      my_coref_list=[]
      my_word_list=[]
      x=mention.split('(')
      number=x[1]
      #print('Case 1',number)
      my_coref_list.append(df['embedding_per_token'][i])
      my_word_list.append(df['tokens_per_word'][i])
      #we call the above method which reads all the mentions
      #until it finds the end of the mention previously encountered
      my_coref_list,my_word_list = add_to_dict(i,number,my_coref_list,my_word_list,df)

    elif '(' in mention and ')' in mention:
      #This is the singleton case
      #Where the mention has both opeining and closing
      my_coref_list=[]
      my_word_list=[]
      x = mention.split('(')
      number=x[1].split(')')[0]
      my_coref_list.append(df['embedding_per_token'][i])
      my_word_list.append(df['tokens_per_word'][i])

    else:
      #print('Case 4 not here')
      continue
    mean_vector=np.mean(my_coref_list,axis=0)
    norm_mean_vector = mean_vector / np.linalg.norm(mean_vector)
    final_df.loc[counter]=[i,number,norm_mean_vector,my_word_list]
    #coref_groups[number] = my_coref_list
    word_groups[number] = my_word_list
    counter+=1

word_groups

{'1': ['the', 'breast', 'cancer', 'susceptibility', 'gene', 'brca2'],
 '4': ['breast', 'cancer', 'susceptibility', 'gene'],
 '5': ['dna'],
 '6': ['dna', 'repair']}

In [36]:
final_df

,index,mention_no,norm_mean_vector,tokens
0,0,4,"[-0.012180175937407974, -0.0026078958793962024...","[two, recent, papers]"
1,11,1,"[-0.0029319319533624697, -0.009076301790759205...","[the, breast, cancer, susceptibility, gene, br..."
2,12,4,"[-0.0035265490149949505, -0.00545502063556827,...","[breast, cancer, susceptibility, gene]"
3,18,6,"[0.009159104415653397, -0.0028121201177927005,...","[dna, repair]"
4,18,5,"[0.006975753121560978, -0.0005256631552159846,...",[dna]


In [28]:
x=final_df['norm_mean_vector'][1]
sum=0
for i in x:
  sum=sum+(i*i)
print(sum)

0.9999999999999997


In [29]:
stacked = np.row_stack(final_df['norm_mean_vector'])
stacked.shape

(5, 768)

In [41]:
all_pairs_cosine_sim = np.dot(stacked,stacked.T)
all_pairs_cosine_sim[0][2]

0.924689110844998

In [ ]:
a='(444aaa)'
if '(' in a and ')' not in a:
  x=a.split('(')
  number=x[1]
  print('Case 1',number)
elif '(' not in a and ')' in a:
  x=a.split(')')
  number = x[0]
  print('Case 2',number)
elif '(' in a and ')' in a:
  x = a.split('(')
  number=x[1].split(')')[0]
  print('Case 3',number)
else:
  print('Case 4 not here')

Case 3 444aaa
